### Imports

In [86]:
%pip install -r dependencies.txt

  Using cached urllib3-2.0.5-py3-none-any.whl.metadata (6.6 kB)
INFO: pip is looking at multiple versions of requests to determine which version is compatible with other requirements. This could take a while.
  Using cached jupyterlab_server-2.20.0-py3-none-any.whl (56 kB)
  Using cached google_api_core-2.12.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached deep_translator-1.11.4-py3-none-any.whl.metadata (30 kB)
  Using cached bardapi-0.1.38-py3-none-any.whl.metadata (21 kB)
ERROR: Cannot install requests==2.28.2 and urllib3==2.0.5 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested urllib3==2.0.5
    requests 2.28.2 depends on urllib3<1.27 and >=1.21.1

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#

In [87]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import requests
import scipy
import importlib
import helper, consts
importlib.reload(consts)
importlib.reload(helper)

<module 'helper' from '/Users/mike/Documents/School/Fa23/Clinic/code/helper.py'>

### CONSTANTS

In [88]:
# USER
USER = "RYAN"
FILE_PATH = consts.PATH_MAP[USER]

ROW = consts.ROW
COL = consts.COL
CLEANED_DATA_PATH = consts.CLEANED_DATA_PATH
DATA_PATH = consts.RAW_DATA_PATH
RESPONSE_NAME = consts.RESPONSE_NAME

TRAIN_START_DATE = "20150101"
TRAIN_END_DATE = "20150601" # Up to but not including
TEST_START_DATE = "20150701"
TEST_END_DATE = "20150801"

In [89]:
REGRESSION_TYPES = helper.Regression('OLS').list_all_regression_types()
REGRESSION_TYPES

1: OLS
2: LASSO
3: XGBOOST


### Read data

In [90]:
train_start, train_end  = helper.get_train_from_testday(TEST_START_DATE)
train_start, train_end
list_of_interacting_terms = [["relvol_nt_0","rrirpnxm_nt_0" ], 
                             ["relvol_lst15_0","rrirpnxm_lst15_0" ],
                             ["relvol_lsthrx15_0","rrirpnxm_lsthrx15_0" ],
                             ["relvol_toxhr_0","rrirpnxm_toxhr_0" ] ]

In [91]:
# x_cols = ["rrirpnxm_nt_0", "rrirpnxm_lst15_0","rrirpnxm_lsthrx15_0", "rrirpnxm_toxhr_0", "relvol_nt_0"]
# train_df = helper.get_df(train_start, train_end, x_cols, FILE_PATH)

train_df = pd.read_csv(consts.CLEANED_DATA_PATH)
pred_cols = list(train_df.columns)
pred_cols.remove(consts.RESPONSE_NAME)
print(pred_cols)


['rrirpnxm_nt_0', 'rrirpnxm_lst15_0', 'rrirpnxm_lsthrx15_0', 'rrirpnxm_toxhr_0', "('relvol_nt_0', 'rrirpnxm_nt_0')", "('relvol_lst15_0', 'rrirpnxm_lst15_0')", "('relvol_lsthrx15_0', 'rrirpnxm_lsthrx15_0')", "('relvol_toxhr_0', 'rrirpnxm_toxhr_0')"]


### Interaction Terms

### Transform the data if needed

#### Ordinary Least Squares

In [92]:
importlib.reload(helper)

ols_regression_model = helper.Regression('OLS')
weights = helper.get_weights(train_df)
model_attributes = ols_regression_model.train(train_df, RESPONSE_NAME, weights)
model_attributes

array([ 4.34327519e-02, -9.87488755e-03, -1.29662094e-03, -2.44584412e-03,
       -1.35271653e-02,  2.23409873e-02, -2.65722018e-03,  4.07870349e-03,
        2.92545274e-05])

In [93]:
importlib.reload(helper)
ols_regression_model.evaluate_model(TEST_START_DATE, TEST_END_DATE, pred_cols,FILE_PATH, list_of_interacting_terms )

Average Weighted Correlation: 0.030368238594772775
Average Mean Return: 0.00022672636319850313
Average weighted Scale Factor: 0.8126962387984625


(0.030368238594772775, 0.00022672636319850313, 0.8126962387984625)

#### OLS w/ Interacting Terms

#### LASSO

In [94]:
lasso_model = helper.Regression('LASSO')
weights = helper.get_weights(train_df)

In [95]:
importlib.reload(helper)
lasso_model.train(train_df, RESPONSE_NAME, weights)
lasso_model.evaluate_model(TEST_START_DATE, TEST_END_DATE, pred_cols, FILE_PATH, list_of_interacting_terms)

Average Weighted Correlation: 0.03017650234224054
Average Mean Return: 0.0002246880217958093
Average weighted Scale Factor: 0.9584213608754041


(0.03017650234224054, 0.0002246880217958093, 0.9584213608754041)

### XGBoost



In [96]:
xgb_model = helper.Regression('XGboost')
weights = helper.get_weights(train_df)
xgb_model.train(train_df, RESPONSE_NAME, weights)
xgb_model.evaluate_model(TEST_START_DATE, TEST_END_DATE, pred_cols, FILE_PATH, list_of_interacting_terms)

/usr/local/lib/python3.10/site-packages/xgboost/core.py:726: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


Average Weighted Correlation: 0.030368238156820052
Average Mean Return: 0.00022672636319850313
Average weighted Scale Factor: 0.8126963973045349


(0.030368238156820052, 0.00022672636319850313, 0.8126964)